In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
#from notebook import tqdm
X_train = df_train[feats]
X_test = df_test[feats]

features = X_train.columns.tolist()
X_train['risk_label'] = 1
X_test['risk_label'] = 0
train_test = pd.concat([X_train, X_test], axis =0,ignore_index=True)
train1, test1 = train_test_split(train_test, test_size=0.33, random_state=42, shuffle=True)
train_y = train1['risk_label'].values
test_y = test1['risk_label'].values
del train1['risk_label'], test1['risk_label']

if 'risk_label' in features:
    features.remove('risk_label')

adversarial_result = pd.DataFrame(index=train1.columns,columns=['roc'])
for i in tqdm(features):
    clf = lgb.LGBMClassifier(
        random_state=47,
        max_depth=2,
        metric='auc',
        n_estimators=1000,
        importance_type='gain'
    )
    clf.fit(
        np.array(train1[i]).reshape(-1,1),
        train_y,
        eval_set=[(np.array(test1[i]).reshape(-1,1),test_y)],
        early_stopping_rounds=200,
        verbose=0)
    temp_pred = clf.predict_proba(np.array(test1[i]).reshape(-1,1))[:,1]
    roc1 = roc_auc_score(test_y,temp_pred)
    adversarial_result.loc[i,'roc'] = roc1
    
a = adversarial_result.sort_values('roc', ascending=False)
a

In [ ]:
import toad
feat_lst = [i for i in df_train.columns if i not in ['LABEL','CUST_UID','MON_12_CUST_CNT_PTY_ID']+drop_fea+yd_col]
psi_df = toad.metrics.PSI(df_train[feat_lst],df_test[feat_lst]).sort_values(0)  
psi_df